In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
df=pd.read_csv("/content/train.csv")
df.head(10)

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5
5,6,2579,132,6,300,-15,67,230,237,140,...,0,0,0,0,0,0,0,0,0,2
6,7,2606,45,7,270,5,633,222,225,138,...,0,0,0,0,0,0,0,0,0,5
7,8,2605,49,4,234,7,573,222,230,144,...,0,0,0,0,0,0,0,0,0,5
8,9,2617,45,9,240,56,666,223,221,133,...,0,0,0,0,0,0,0,0,0,5
9,10,2612,59,10,247,11,636,228,219,124,...,0,0,0,0,0,0,0,0,0,5


In [8]:
df.columns

Index(['Id', 'Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points',
       'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3',
       'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3',
       'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8',
       'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12',
       'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16',
       'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20',
       'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24',
       'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28',
       'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32',
       'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36',
       'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_

In [9]:
df.drop(columns='Id',errors='ignore')
num_features = [
    "Elevation", "Aspect", "Slope",
    "Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology",
    "Horizontal_Distance_To_Roadways", "Hillshade_9am",
    "Hillshade_Noon", "Hillshade_3pm", "Horizontal_Distance_To_Fire_Points"
]

scaler=StandardScaler()
df[num_features]=scaler.fit_transform(df[num_features])
X=df.drop(columns='Cover_Type')
Y=df["Cover_Type"]


In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.4,random_state=42,stratify=Y)

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
model1=RandomForestClassifier(n_estimators=100,random_state=42)
model1.fit(X_train,Y_train)
Y_pred=model1.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)

In [12]:
accuracy

0.8583002645502645

In [13]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

rf = RandomForestClassifier(random_state=42)
grid_search = RandomizedSearchCV(rf, param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, Y_train)

# Best parameters
print("Best Parameters:", grid_search.best_params_)

# Best model
best_model = grid_search.best_estimator_


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Parameters: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 20}


In [14]:
best_model=RandomForestClassifier(n_estimators=200,min_samples_split=5,min_samples_leaf=1,max_depth=20,random_state=42,class_weight="balanced")
best_model.fit(X_train,Y_train)
Y_pred=best_model.predict(X_test)


In [15]:
from sklearn.metrics import accuracy_score, classification_report
print("Test Accuracy:", accuracy_score(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

Test Accuracy: 0.8520171957671958
              precision    recall  f1-score   support

           1       0.78      0.76      0.77       864
           2       0.81      0.67      0.73       864
           3       0.82      0.80      0.81       864
           4       0.92      0.98      0.95       864
           5       0.89      0.94      0.91       864
           6       0.82      0.86      0.84       864
           7       0.91      0.97      0.94       864

    accuracy                           0.85      6048
   macro avg       0.85      0.85      0.85      6048
weighted avg       0.85      0.85      0.85      6048



In [16]:
importances = best_model.feature_importances_
feature_names = X_train.columns
sorted_indices = importances.argsort()[::-1]
for idx in sorted_indices[:10]:  # Top 10 features
    print(f"{feature_names[idx]}: {importances[idx]:.4f}")


Elevation: 0.2201
Horizontal_Distance_To_Roadways: 0.0819
Id: 0.0764
Horizontal_Distance_To_Fire_Points: 0.0601
Wilderness_Area4: 0.0541
Horizontal_Distance_To_Hydrology: 0.0536
Vertical_Distance_To_Hydrology: 0.0446
Hillshade_9am: 0.0445
Aspect: 0.0412
Hillshade_Noon: 0.0372


In [17]:
from sklearn.ensemble import RandomForestClassifier
best_model = RandomForestClassifier(n_estimators=100, random_state=42)
best_model.fit(X_train, Y_train)

RandomForestClassifier(random_state=42)

In [19]:
Y_pred = best_model.predict(X_test)  # Predictions for the test set

In [20]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(Y_test, Y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.86


In [21]:
from sklearn.metrics import confusion_matrix, classification_report

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(Y_test, Y_pred))

# Detailed Report (Precision, Recall, F1-score)
print("\nClassification Report:")
print(classification_report(Y_test, Y_pred))

Confusion Matrix:
[[666 115   2   0  13   1  67]
 [162 581  22   0  61  32   6]
 [  0   4 704  49  10  97   0]
 [  0   0  14 840   0  10   0]
 [  2  26  16   0 809  11   0]
 [  0   5  89  17   3 750   0]
 [ 22   1   0   0   0   0 841]]

Classification Report:
              precision    recall  f1-score   support

           1       0.78      0.77      0.78       864
           2       0.79      0.67      0.73       864
           3       0.83      0.81      0.82       864
           4       0.93      0.97      0.95       864
           5       0.90      0.94      0.92       864
           6       0.83      0.87      0.85       864
           7       0.92      0.97      0.95       864

    accuracy                           0.86      6048
   macro avg       0.86      0.86      0.86      6048
weighted avg       0.86      0.86      0.86      6048

